**Mounting colab on drive**

In [2]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Hate Video task'
assert FOLDERNAME is not None, "[!] Enter the foldername."

Mounted at /content/drive





**Unzipping the rar file**


In [ ]:
source = '/content/drive/My Drive/Hate Video task/Dataset.rar/'
destination = '/content/drive/My Drive/Hate Video task/Dataset_alt/'
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(source).extractall(destination)

**Data set creation**



In [3]:
import cv2
import math
from tqdm import tqdm
from glob import glob
import numpy as np
import pandas as pd
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import os

In [ ]:
data_folder = '/content/drive/My Drive/Hate Video task/Dataset/Training_videos/'



for i in tqdm(range(1,6)):
  train_images = []
  train_class = []
  destination_folder = '/content/drive/My Drive/Hate Video task/Dataset/Training_images/Training_images' + str(i) + '/'
  # Folder 1
  cnt = 1;
  f1 = data_folder + 'hate_videos' + str(i) + '/'
  for entry in os.listdir(f1):
    fin_dest = destination_folder + 'Vid' + str(cnt) + '/'
    os.mkdir(fin_dest)
    count = 0
    file_name = f1 + str(entry)
    cap = cv2.VideoCapture(file_name)
    frame_rate = cap.get(5)
    x = 1
    while(cap.isOpened()):
      frame_id = cap.get(1)
      ret, frame = cap.read()
      if(ret!=True):
        break
      if(frame_id%math.floor(frame_rate)==0):
        file_name = fin_dest + 'img' + str(count) + '.jpg'
        count+=1
        train_images.append(file_name)
        train_class.append(1)
        cv2.imwrite(file_name, frame)
    cap.release()
    cnt+=1

  print("Done 1/2")

  #Folder 2
  f1 = data_folder + 'normal_videos' + str(i) + '/'
  for entry in os.listdir(f1):
    fin_dest = destination_folder + 'Vid' + str(cnt) + '/'
    os.mkdir(fin_dest)
    count = 0
    file_name = f1 + str(entry)
    cap = cv2.VideoCapture(file_name)
    frame_rate = cap.get(5)
    x = 1
    while(cap.isOpened()):
      frame_id = cap.get(1)
      ret, frame = cap.read()
      if(ret!=True):
        break
      if(frame_id%math.floor(frame_rate)==0):
        file_name = fin_dest + 'img' + str(count) + '.jpg'
        count+=1
        train_images.append(file_name)
        train_class.append(0)
        cv2.imwrite(file_name, frame)
    cap.release()
    cnt+=1

  print("Done 2/2")

  data_set = pd.DataFrame()
  data_set['Images'] = train_images
  data_set['Classes'] = train_class

  csv_name = '/content/drive/My Drive/Hate Video task/Dataset/train_data' + str(i) + '.csv'
  data_set.to_csv(csv_name,header=True, index=False)



  0%|          | 0/5 [00:00<?, ?it/s]

Done 1/2




 20%|██        | 1/5 [02:00<08:01, 120.38s/it]

Done 2/2
Done 1/2




 40%|████      | 2/5 [04:14<06:13, 124.44s/it]

...

Done 2/2
Done 1/2




 80%|████████  | 4/5 [08:22<02:04, 124.20s/it]

Done 2/2
Done 1/2




100%|██████████| 5/5 [11:47<00:00, 141.43s/it]

Done 2/2


**Test data set creation**

In [ ]:
data_folder = '/content/drive/My Drive/Hate Video task/Dataset/'
destination_folder = '/content/drive/My Drive/Hate Video task/Dataset/Test_images/'

test_images = []
test_class = []

# Folder 1
cnt = 1;
f1 = data_folder + 'Test_hate/'
for entry in os.listdir(f1):
  fin_dest = destination_folder + 'Vid' + str(cnt) + '/'
  os.mkdir(fin_dest)
  count = 0
  file_name = f1 + str(entry)
  cap = cv2.VideoCapture(file_name)
  frame_rate = cap.get(5)
  x = 1
  while(cap.isOpened()):
    frame_id = cap.get(1)
    ret, frame = cap.read()
    if(ret!=True):
      break
    if(frame_id%math.floor(frame_rate)==0):
      file_name = fin_dest + 'img' + str(count) + '.jpg'
      count+=1
      test_images.append(file_name)
      test_class.append(1)
      cv2.imwrite(file_name, frame)
  cap.release()
  cnt+=1

print("Done 1/2")

#Folder 2
f1 = data_folder + 'Test_normal/'
for entry in os.listdir(f1):
  fin_dest = destination_folder + 'Vid' + str(cnt) + '/'
  os.mkdir(fin_dest)
  count = 0
  file_name = f1 + str(entry)
  cap = cv2.VideoCapture(file_name)
  frame_rate = cap.get(5)
  x = 1
  while(cap.isOpened()):
    frame_id = cap.get(1)
    ret, frame = cap.read()
    if(ret!=True):
      break
    if(frame_id%math.floor(frame_rate)==0):
      file_name = fin_dest + 'img' + str(count) + '.jpg'
      count+=1
      test_images.append(file_name)
      test_class.append(0)
      cv2.imwrite(file_name, frame)
  cap.release()
  cnt+=1

print("Done 2/2")


data_set = pd.DataFrame()
data_set['Images'] = test_images
data_set['Classes'] = test_class

csv_name = '/content/drive/My Drive/Hate Video task/Dataset/test_data.csv'
data_set.to_csv(csv_name,header=True, index=False)

Done 1/2
Done 2/2


**Audio Extraction from video**

In [7]:
import librosa
import moviepy.editor
data_folder = '/content/drive/My Drive/Hate Video task/Dataset/Test_video/'
dest_folder = '/content/drive/My Drive/Hate Video task/Dataset/Test_audio/'
for entry in os.listdir(data_folder):
  video = moviepy.editor.VideoFileClip(data_folder+entry)
  audio = video.audio
  f_name = entry.split('.')[0]+'.wav'
  audio.write_audiofile(dest_folder+f_name)

[MoviePy] Writing audio in /content/drive/My Drive/Hate Video task/Dataset/Test_audio/normal (104).wav


100%|██████████| 66/66 [00:00<00:00, 1746.52it/s]

[MoviePy] Done.
[MoviePy] Writing audio in /content/drive/My Drive/Hate Video task/Dataset/Test_audio/normal (172).wav



100%|██████████| 27/27 [00:00<00:00, 1875.12it/s]

[MoviePy] Done.


...

100%|██████████| 1336/1336 [00:00<00:00, 1748.09it/s]

[MoviePy] Done.


[MoviePy] Writing audio in /content/drive/My Drive/Hate Video task/Dataset/Test_audio/hate (83).wav


100%|██████████| 1301/1301 [00:00<00:00, 1866.37it/s]

[MoviePy] Done.
